# Using an output handler for validating agent's output

As no one can guarantee the validity or integrity of a raw LLM output, getting a sensible and/or structured output from an agent is tricky. In motleycrew, we addressed this issue with a novel approach: please welcome **output handlers**.

As of now, tool calling is the most reliable way to get a structured output from an LLM. So why not have an output handler - a regular tool with an input schema and a description which instructs the agent to return the output only via this tool. Moreover, we can force the agent into doing this: if it attempts to finish the regular way, we intercept the message and remind the agent that it must call the tool instead.

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

from motleycrew import MotleyCrew
from motleycrew.agents.langchain import ReActToolCallingMotleyAgent
from motleycrew.common import configure_logging
from motleycrew.tasks import SimpleTask

from motleycrew.common.exceptions import InvalidOutput

from langchain_core.tools import StructuredTool

We'll be prompting our agent to create a report on the latest advancements in AI.  
Say we want it to also include some info on AI applications in physics if it did not already. So our output handler will look for the keyword "physics" in the agent's output.

Also, we can do any data manipulations we want inside the output handler. In this case, let's just place the output into a certain field.

In [2]:
def check_output(output: str):
    if "physics" not in output.lower():
        raise InvalidOutput("Add more information about AI applications in physics.")

    return {"checked_output": output}

output_handler = StructuredTool.from_function(
    name="output_handler",
    description="Output handler",
    func=check_output,
)  # Schema is inferred from the function signature, but you can also specify it explicitly

Now let's create our agent and launch the crew!

In [3]:
search_tool = DuckDuckGoSearchRun()
researcher = ReActToolCallingMotleyAgent(
    tools=[search_tool],
    verbose=True,
    chat_history=True,
    output_handler=output_handler,
)

crew = MotleyCrew()
task = SimpleTask(
    crew=crew,
    name="produce comprehensive analysis report on AI advancements in 2024",
    description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report""",
    agent=researcher,
)

2024-07-19 17:06:03,116 - motleycrew - WARNING - No known Cypher type matching annotation typing.Optional[typing.Any], will use JSON string
2024-07-19 17:06:03,120 - motleycrew - WARNING - No known Cypher type matching annotation typing.Optional[dict[str, typing.Any]], will use JSON string


In [4]:
crew.run()

2024-07-19 17:06:03,183 - motleycrew - WARNING - No known Cypher type matching annotation typing.Optional[typing.Any], will use JSON string
2024-07-19 17:06:03,183 - motleycrew - WARNING - No known Cypher type matching annotation typing.Optional[dict[str, typing.Any]], will use JSON string




> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'latest advancements in AI 2024'}`
responded: Thought: To conduct a comprehensive analysis of the latest advancements in AI in 2024, I need to gather information on key trends, breakthrough technologies, and potential industry impacts. I will start by searching for the latest news and reports on AI advancements in 2024.

I will use the DuckDuckGo search tool to find relevant information.



Here are some important current AI trends to look out for in the coming year. Reality check: more realistic expectations. Multimodal AI. Small (er) language models and open source advancements. GPU shortages and cloud costs. Model optimization is getting more accessible. Customized local models and data pipelines. More powerful virtual agents. In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinkering with a million little AI models. State-of

2024-07-19 17:07:33,737 - motleycrew - WARNING - No known Cypher type matching annotation typing.Optional[typing.Any], will use JSON string



Invoking: `output_handler` with `{'output': '### Comprehensive Analysis of AI Advancements in 2024\n\n#### Key Trends in AI 2024\n\n1. **Generative AI Maturation**:\n   - Generative AI, which gained significant attention in 2023, is expected to become more practical and useful for everyday users in 2024. This includes applications in content creation, customer service, and personalized experiences.\n   - The adoption of generative AI by businesses is accelerating, with 65% of organizations regularly using it, nearly double the percentage from the previous year.\n\n2. **Multimodal AI**:\n   - Multimodal AI, which processes and integrates data from multiple sources (e.g., text, images, audio), is becoming more prevalent. This allows for more sophisticated and context-aware AI applications.\n\n3. **Smaller and Open Source Models**:\n   - There is a trend towards smaller, more efficient AI models that are easier to deploy and manage. Open source AI models are gaining traction, enabling bu

[TaskUnit(status=done)]

As you can see in the logs, our physics requirement was successfully fed back to the agent!

In [5]:
from IPython.display import display, Markdown

final_result = task.output["checked_output"]
display(Markdown(final_result))

### Comprehensive Analysis of AI Advancements in 2024

#### Key Trends in AI 2024

1. **Generative AI Maturation**:
   - Generative AI, which gained significant attention in 2023, is expected to become more practical and useful for everyday users in 2024. This includes applications in content creation, customer service, and personalized experiences.
   - The adoption of generative AI by businesses is accelerating, with 65% of organizations regularly using it, nearly double the percentage from the previous year.

2. **Multimodal AI**:
   - Multimodal AI, which processes and integrates data from multiple sources (e.g., text, images, audio), is becoming more prevalent. This allows for more sophisticated and context-aware AI applications.

3. **Smaller and Open Source Models**:
   - There is a trend towards smaller, more efficient AI models that are easier to deploy and manage. Open source AI models are gaining traction, enabling businesses to customize and optimize AI solutions for their specific needs.

4. **Ethics and Regulation**:
   - As AI technologies become more integrated into society, there is a growing emphasis on ethical considerations, safety, and regulatory compliance. Organizations are adopting more cautious and responsible AI development and deployment strategies.

5. **AI in Data and Analytics**:
   - The role of AI in data and analytics is evolving, with a focus on integrating AI capabilities into existing data pipelines and analytics workflows. This trend is driven by the need for more accurate and actionable insights.

#### Breakthrough AI Technologies in 2024

1. **AI-Powered Scientific Discovery**:
   - AI is playing a crucial role in accelerating scientific research and discovery. AI algorithms are being used to analyze vast amounts of data, identify patterns, and generate new hypotheses in fields such as medicine, materials science, and climate research.

2. **Elastocalorics**:
   - Elastocaloric materials, which can change temperature under mechanical stress, are being explored for their potential in energy-efficient cooling systems. AI is being used to optimize the design and performance of these materials.

3. **Advanced Virtual Agents**:
   - Virtual agents are becoming more powerful and capable of handling complex interactions. These agents are being used in customer service, healthcare, and education to provide personalized and efficient support.

4. **Model Optimization**:
   - Techniques for optimizing AI models, such as pruning and quantization, are becoming more accessible. This allows for the deployment of AI models on resource-constrained devices, such as smartphones and IoT devices.

5. **AI in Physics**:
   - Researchers are using generative AI to develop physics-informed techniques for classifying phase transitions in materials and physical systems. This approach is more efficient than existing machine-learning methods and has the potential to drive new innovations in various industries.

#### Potential Industry Impacts

1. **Manufacturing**:
   - The manufacturing industry is expected to see the largest financial impact from AI. AI-driven automation, predictive maintenance, and quality control are enhancing productivity and reducing costs.

2. **Healthcare**:
   - AI is transforming healthcare by enabling early diagnosis, personalized treatment plans, and efficient management of healthcare resources. AI-powered tools are improving patient outcomes and reducing the burden on healthcare professionals.

3. **Finance**:
   - In the finance sector, AI is being used for fraud detection, risk management, and personalized financial services. AI algorithms are enhancing the accuracy and efficiency of financial operations.

4. **Retail**:
   - AI is revolutionizing the retail industry by enabling personalized shopping experiences, optimizing supply chain management, and enhancing customer service. Retailers are using AI to analyze consumer behavior and predict trends.

5. **Energy**:
   - AI is playing a key role in optimizing energy production and consumption. AI algorithms are being used to manage smart grids, predict energy demand, and improve the efficiency of renewable energy sources.